In [13]:
import pandas as pd
import utils
import io
import datetime
pd.options.display.max_columns=999

In [9]:
import pandas as pd
# 2016
# date1 = '2016-04-02'
# date2 = '2016-10-02'
# 2017
date1 = '2017-04-02'
date2 = '2017-10-01'
# 2018
# date1 = '2018-03-29'
# date2 = '2018-10-01'
mydates = pd.date_range(date1, date2).tolist()
BASE_URL="http://rotoguru1.com/cgi-bin/byday.pl?game=dk&month=MONTH&day=DAY&year=YEAR" #Draftkings

In [10]:
positions_dict = dict()
positions_dict["2"] = "C"
positions_dict["3"] = "1B"
positions_dict["4"] = "2B"
positions_dict["5"] = "3B"
positions_dict["6"] = "SS"
positions_dict["7"] = "OF"

In [11]:
for date in mydates:
    result = pd.to_datetime(date)
    result += datetime.timedelta(days=1)
    date_str=str(result.month) + "_" + str(result.day) + "_" + str(result.year)
    BASE_URL="http://rotoguru1.com/cgi-bin/byday.pl?game=dk&month=MONTH&day=DAY&year=YEAR" #Draftkings\
    soup=utils.soup(BASE_URL.replace("DAY", str(date.day)).replace("MONTH", str(date.month)).replace("YEAR", str(date.year)))
    lines = soup.get_text().split('\n')
    pitchers, batters, done = False, False, False
    df = pd.DataFrame(columns=['Position', 'Name', 'Points', 'Salary'])
    for line in lines:
        if done:
            continue
        if "Jump to" in line:
            if pitchers:
                batters=True
                continue
            pitchers=True
            continue
        if "For data in semi-colon delimited text format, click here." in line and batters is True:
            done = True
        if pitchers and not batters:
            parts = line.split(" ")
            parts = list(filter(None, parts))
            print(parts)
            name, end = "", 0
            
            # Get the names built
            while "^" not in parts[end] and not (end == len(parts)-1):
                name = name + " " + parts[end]
                end = end + 1
            name = name + " " + parts[end].split("^")[0]
            try:
                order = parts[end].split("^")[1]
            except Exception:
                order = "0"
            # Special cases in the data
            if (end == len(parts)-1):
                continue
            for i in range(1, end):
                parts.pop(i)
            if "-" in parts[1]:
                opponent = parts[2][:3]
                position = parts[0][0]
                if "$" in parts[1]:
                    last = parts[1].split("^")[0]
                    parts = parts[1].split("^")[1]
                    order = parts.split("-")[0]
                    parts = parts.split("-")[1]
                    points = "-" + parts.split("$")[0]
                    parts = parts.split("$")[1]
                    salary = parts[:-4]
                    team = parts[-4:-1]
                    df = df.append({'Position': position,
                                    'Name': name[2:],
                                    'Points':points,
                                    'Salary':salary,
                                    'Order': order,
                                    'Team': team,
                                    'Opponent': opponent}, ignore_index=True)
                elif "Bibens-Dirkx^0" in parts[1]:
                    df = df.append({'Position': parts[0][0],
                            'Name': name[2:],
                            'Points':parts[2].split("$")[0],
                            'Salary':parts[2].split("$")[1][:-4],
                            'Order': order,
                            'Team': parts[2].split("$")[1][-4:-1],
                            'Opponent': parts[3][:3]}, ignore_index=True)
                continue
            if "N/A" in parts[2]:
                ## Skip ones without salary data, no reason to add them for now
                continue
            # Add the player
            df = df.append({'Position': parts[0][0],
                            'Name': name[2:],
                            'Points':parts[2].split("$")[0],
                            'Salary':parts[2].split("$")[1][:-4],
                            'Order': order,
                            'Team': parts[2].split("$")[1][-4:-1],
                            'Opponent': parts[3][:3]}, ignore_index=True)

        if pitchers and batters and not done:
            parts = line.split(" ")
            parts = list(filter(None, parts))
            print(parts)
            if "N/A" in parts[2]:
                continue
            # Get the names built
            name, end = "", 0
            while "^" not in parts[end] and not (end == len(parts)-1):
                name = name + " " + parts[end]
                end = end + 1
            name = name + " " + parts[end].split("^")[0]
            try:
                order = parts[end].split("^")[1]
            except Exception:
                order = "0"
                
            if (end == len(parts)-1):
                continue
            # remove middle name characters for easier parsing
            for i in range(1, end):
                parts.pop(i)
            # Find where the positional numbers are gone
            if "N/A" in parts[2]:
                continue
            pos_num = 0
            while parts[0][pos_num].isdigit():
                pos_num = pos_num+1
            if pos_num == 0:
                pos = ""
            else:
                pos = positions_dict[parts[0][0]]
            # Add the player
            df = df.append({'Position': pos,
                            'Name': name[pos_num+1:],
                            'Points':parts[2].split("$")[0], 
                            'Salary':parts[2].split("$")[1][:-4],
                            'Order': order,
                            'Team': parts[2].split("$")[1][-4:-1],
                            'Opponent': parts[3][:3]}, ignore_index=True)
            
    df.to_csv("2017_update/" + date_str + ".csv", index=False)

AttributeError: module 'utils' has no attribute 'soup'